In [321]:
import requests
import pandas as pd
import datetime
from datetime import datetime
import numpy as np
import sqlite3

In [322]:
url_groupdetail = "https://api.purpleair.com/v1/groups/1662"
url_newmember = "https://api.purpleair.com/v1/groups/1662/members"
url_sensordata = "https://api.purpleair.com/v1/sensors/39355"
url_groupdata = "https://api.purpleair.com/v1/groups/1662/members"
url_deletemember = "https://api.purpleair.com/v1/groups/1662/members/158862"

key_get = "C111944A-B3A7-11ED-B6F4-42010A800007"
key_post = "C112A231-B3A7-11ED-B6F4-42010A800007"
headers = {'X-API-Key': key_get}

### List of sensors in San Pedro

In [323]:
sensors = requests.get(url_groupdetail, headers=headers)

In [324]:
sensors = sensors.json()

In [325]:
sensors = sensors['members']

In [326]:
sensors

[{'id': 158850, 'sensor_index': 34641, 'created': 1677195580},
 {'id': 158851, 'sensor_index': 37747, 'created': 1677196464},
 {'id': 158849, 'sensor_index': 38537, 'created': 1677195516},
 {'id': 158854, 'sensor_index': 143372, 'created': 1677196569},
 {'id': 160198, 'sensor_index': 39355, 'created': 1677264653},
 {'id': 158855, 'sensor_index': 39697, 'created': 1677196598},
 {'id': 158852, 'sensor_index': 34569, 'created': 1677196500},
 {'id': 158853, 'sensor_index': 39731, 'created': 1677196526},
 {'id': 158858, 'sensor_index': 147010, 'created': 1677196677},
 {'id': 158859, 'sensor_index': 140440, 'created': 1677196694},
 {'id': 158856, 'sensor_index': 173083, 'created': 1677196625},
 {'id': 158857, 'sensor_index': 45769, 'created': 1677196650},
 {'id': 158863, 'sensor_index': 121825, 'created': 1677196842},
 {'id': 158860, 'sensor_index': 34511, 'created': 1677196728},
 {'id': 158861, 'sensor_index': 49563, 'created': 1677196755},
 {'id': 158866, 'sensor_index': 37657, 'created': 

In [327]:
sensors = pd.DataFrame(sensors)

In [328]:
sensors = sensors[['sensor_index', 'id', 'created']]

sensors.rename(columns = {'sensor_index':'index'}, inplace = True)


In [329]:
sensors.rename(columns = {'id': 'group_id'}, inplace = True)

In [330]:
sensors = sensors.drop('created', axis=1)

In [331]:
sensors.head()

,index,group_id
0,34641,158850
1,37747,158851
2,38537,158849
3,143372,158854
4,39355,160198


### PM 2.5

In [332]:
payload_pm25 = {'fields': 'pm2.5'}

In [333]:
pm25 = requests.get(url_groupdata, headers = headers, params = payload_pm25)

In [334]:
pm25 = pm25.json()

In [335]:
pm25 = pm25['data']

In [336]:
pm25 = pd.DataFrame(pm25, columns = ('index', 'pm_25'))

In [337]:
pm25.head()

,index,pm_25
0,140432,19.6
1,140440,22.1
2,143372,22.1
3,147010,17.6
4,33117,18.6


### PM 10

In [338]:
payload_pm10 = {'fields': 'pm10.0'}

In [339]:
pm10 = requests.get(url_groupdata, headers = headers, params = payload_pm10)

In [340]:
pm10 = pm10.json()

In [341]:
pm10 = pm10['data']

In [342]:
pm10 = pd.DataFrame(pm10, columns = ('index', 'pm_10'))

In [343]:
pm10.head()

,index,pm_10
0,140432,23.4
1,140440,22.7
2,143372,28.7
3,147010,19.6
4,33117,15.7


### Temperature

In [344]:
payload_temp = {'fields': 'temperature'}

In [345]:
temp = requests.get(url_groupdata, headers = headers, params = payload_temp)

In [346]:
temp = temp.json()

In [347]:
temp = temp['data']

In [348]:
temp = pd.DataFrame(temp, columns = ('index', 'temperature_f'))

In [349]:
# Convert fahrenheit to celsius.
temp['temp_celsius'] = np.round((temp["temperature_f"] - 32) * .5556, decimals = 0)
temp['temp_celsius'] = temp['temp_celsius'].astype(int)

# Delete fahrenheit column.
temp = temp.drop('temperature_f', axis=1)

In [350]:
temp.head()

,index,temp_celsius
0,140432,31
1,140440,33
2,143372,34
3,147010,37
4,33117,36


### Date

In [351]:
now = datetime.now()

In [352]:
timestamp = now.strftime("%d/%m/%Y %H:%M:%S")

In [353]:
timestamp = pd.Series(timestamp)

In [354]:
timestamp = pd.DataFrame(timestamp)

In [355]:
timestamp = pd.DataFrame(np.repeat(timestamp.values, 32, axis=0))

In [356]:
timestamp.head()

,0
0,28/02/2023 12:04:11
1,28/02/2023 12:04:11
2,28/02/2023 12:04:11
3,28/02/2023 12:04:11
4,28/02/2023 12:04:11


### Join data into one database

In [357]:
air_pollution = pd.concat([sensors.set_index('index'),pm25.set_index('index')], axis=1, join='inner').reset_index()

In [358]:
air_pollution = pd.concat([air_pollution.set_index('index'), pm10.set_index('index')], axis=1, join='inner').reset_index()

In [359]:
air_pollution = pd.concat([air_pollution.set_index('index'), temp.set_index('index')], axis=1, join='inner').reset_index()

In [360]:
air_pollution['date'] = timestamp

In [361]:
air_pollution

,index,group_id,pm_25,pm_10,temp_celsius,date
0,34641,158850,18.6,21.5,33,28/02/2023 12:04:11
1,37747,158851,18.4,22.5,31,28/02/2023 12:04:11
2,38537,158849,23.5,28.6,34,28/02/2023 12:04:11
3,143372,158854,22.1,28.7,34,28/02/2023 12:04:11
4,39355,160198,19.8,28.1,33,28/02/2023 12:04:11
5,39697,158855,20.2,23.4,32,28/02/2023 12:04:11
6,34569,158852,17.0,18.9,37,28/02/2023 12:04:11
7,39731,158853,19.0,18.6,37,28/02/2023 12:04:11
8,147010,158858,17.6,19.6,37,28/02/2023 12:04:11
9,140440,158859,22.1,22.7,33,28/02/2023 12:04:11


In [362]:
air_pollution.dtypes

index             int64
group_id          int64
pm_25           float64
pm_10           float64
temp_celsius      int64
date             object
dtype: object

### Create sql database

In [363]:
conn = sqlite3.connect('air-pollution-prueba.db')
c = conn.cursor()

c.execute('''
          CREATE TABLE IF NOT EXISTS air_pollution
          ([index] INTEGER PRIMARY KEY, [group_id] INTEGER, [pm_25] REAL, [pm_10] REAL, [temp_celsius] INTEGER,
          [date] TEXT)
          ''')

conn.commit()

In [364]:
#conn = sqlite3.connect('air-pollution-prueba.db')
#c = conn.cursor()

#air_pollution.to_sql('air_pollution', conn, if_exists='append', index=True)

#conn.commit()

### SQL Database

In [365]:
conn = sqlite3.connect('air-pollution-prueba.db') 
c = conn.cursor()
                   
c.execute('''
          SELECT
          index, group_id, pm_25, pm_10, temp_celsius, date FROM air_pollution;
          ''')

df = pd.DataFrame(c.fetchall(), columns=['index','group_id', 'pm_25', 'pm_10', 'temp_celsius', 'date'])
df

OperationalError: near "index": syntax error